In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import chi2_contingency
from matplotlib.ticker import PercentFormatter
from sklearn.model_selection import train_test_split, GridSearchCV
import imblearn
plt.rcParams["patch.force_edgecolor"] = True
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from scipy import stats

In [ ]:
# Import the data into a dataframe
test = pd.read_csv('data/Test.csv')
train = pd.read_csv('data/train.csv')

test = test.drop('uniqueid', axis=1)
train = train.drop('uniqueid', axis=1)

In [ ]:
train.head()

In [ ]:
# Make bins for household size in train
household_bins = pd.cut(train['household_size'], [1, 3, 7, 10, 25], labels=['single', 'small', 'average', 'big'])
household_bins.name = 'household_sizes'

train = train.join(household_bins, how='inner')
train = train.drop('household_size', axis=1)
train.head()

In [ ]:
# Make bins for household size in test
household_bins = pd.cut(test['household_size'], [1, 3, 7, 10, 25], labels=['single', 'small', 'average', 'big'])
household_bins.name = 'household_sizes'

test = test.join(household_bins, how='inner')
test = test.drop('household_size', axis=1)
test.head()

In [ ]:
# Make bins for age in train
age_bins = pd.cut(train['age_of_respondent'], [0, 16, 45, 75, 100], labels=['child', 'adult', 'elder', 'old'])
age_bins.name = 'ages'

train = train.join(age_bins, how='inner')
train = train.drop('age_of_respondent', axis=1)
train.head()

In [ ]:
# Make bins for age in test
age_bins = pd.cut(test['age_of_respondent'], [0, 16, 45, 75, 100], labels=['child', 'adult', 'elder', 'old'])
age_bins.name = 'ages'

test = test.join(age_bins, how='inner')
test = test.drop('age_of_respondent', axis=1)
test.head()

In [ ]:
# make dummie variables for train
train = pd.get_dummies(train, drop_first=True)
train.head().T

In [ ]:
# make dummie variables for test
test = pd.get_dummies(test, drop_first=True)
test.head().T

In [ ]:
# Train-test Split

# Defining X and y
features = train.columns.tolist()
features.remove('bank_account_Yes')

X = train[features]
y = train.bank_account_Yes

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True, stratify=y)

# Check the shape of the data sets
print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_test:", X_test.shape)
print("y_test:", y_test.shape)

In [ ]:
# Perform undersampling on the majority class
undersampler = RandomUnderSampler(random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [ ]:
# Perform oversampling on the minority class
oversampler = SMOTE(random_state=42)
X_train_over, y_train_over = oversampler.fit_resample(X_train, y_train)

## Decision Tree

In [ ]:
# Create Decision Tree
dtree = DecisionTreeClassifier()
dtree.fit(X_train, y_train)

In [ ]:
# Predict
y_pred_tree = dtree.predict(X_test)
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

In [ ]:
# Create Decision Tree with undersampling
dtree = DecisionTreeClassifier()
dtree.fit(X_train_under, y_train_under)

In [ ]:
# Predict with undersampling
y_pred_tree = dtree.predict(X_test)
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

In [ ]:
# Create Decision Tree with oversampling
dtree = DecisionTreeClassifier()
dtree.fit(X_train_over, y_train_over)

In [ ]:
# Predict with oversampling
y_pred_tree = dtree.predict(X_test)
print(confusion_matrix(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))

---

## Random Forest

In [ ]:
#Create Random Forest
forest = RandomForestClassifier()
forest.fit(X_train, y_train)

In [ ]:
# Predict
y_pred_forest = dtree.predict(X_test)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

In [ ]:
#Create Random Forest with undersampling
forest = RandomForestClassifier()
forest.fit(X_train_under, y_train_under)

In [ ]:
# predict with undersampling
y_pred_forest = dtree.predict(X_test)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

In [ ]:
#Create Random Forest with oversampling
forest = RandomForestClassifier()
forest.fit(X_train_over, y_train_over)

In [ ]:
# Predict with oversampling
y_pred_forest = dtree.predict(X_test)
print(y_pred_forest)
print(confusion_matrix(y_test, y_pred_forest))
print(classification_report(y_test, y_pred_forest))

---

### Decision Tree w/o over- or undersampling has best precision
__Fit test data to model__

In [ ]:
test.head().T

In [ ]:
# Defining X and  for test
features = test.columns.tolist()
X = test[features]

# Predict
y_pred_tree = dtree.predict(X)
series = pd.Series(y_pred_tree).astype(int)
series.name = 'bank_account'

test_new = pd.concat([test, series], axis=1).astype(bool)
test_new.head().T

In [ ]:
# make countplot for bank_account of train dataset
# Countplot with bank account, hue=gender
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(x=train['bank_account_Yes'], hue=train['gender_of_respondent_Male']);

In [ ]:
# make countplot for bank_account of test dataset
# Countplot with bank account, hue=gender
sns.set(rc={'figure.figsize':(11.7,8.27)})
sns.countplot(x=test_new['bank_account'], hue=test_new['gender_of_respondent_Male']);